In [ ]:
#Bayesian modelling for Plasmodium relictum
import pystan
import pickle
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import arviz as az

np.random.seed(2001)

#Stan code to estimate value of unknown parameter - maximum temperature (Tmax),
#minimum temperature (Tmin) and scaling parameter (c)
model="""
data {
      int<lower=0> N;
      vector[N] x;
      vector[N] y;
}
parameters {
    real<lower=31,upper=38> Tmax;
    real<lower=0,upper=15> Tmin;
    real <lower=0>c;
    real <lower=0> tau;
 }  
transformed parameters{
    real<lower=0> sigma;
    real<lower=0> pdr[N];
    real<lower=0> Tmax_1;
    real<lower=0> Tmin_1;
    real<lower=0> c_1;
    Tmax_1=Tmax;
    Tmin_1=Tmin;
    c_1=c;
    for(i in 1:N){
      pdr[i]=c_1*x[i]*(x[i]-Tmin_1)*sqrt(Tmax_1-x[i]);
    }
    sigma=1.0/tau;
} 
model {
      Tmax ~ uniform(31,38);
      Tmin ~ uniform(0,15);
      c ~ gamma(1,10);
      tau ~ gamma(0.0001,0.0001);
      y ~ normal(pdr,sigma);
   
}        
"""
#Input data for the temperature
x=np.array([15,17,19,20,21,25,28])

#Temperature dependant parasite development rate
y=np.array([0.0285,0.0384,0.0588,0.0909,0.1111,0.1428,0.1667])

data = {'N': len(x), 'x': x, 'y': y}

sm=pystan.StanModel(model_code=model)
fit = sm.sampling(data=data, iter=1000, chains=4, warmup=500, thin=1, seed=2001,verbose=True)
print(fit)